# OLS (Linear Regression)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf

from plotnine import *
%matplotlib inline

In [ ]:
url = 'https://raw.githubusercontent.com/mwaugh0328/Data_Bootcamp_Fall_2017/master/data_bootcamp_1127/trump_data.csv'
df = pd.read_csv(url,encoding='latin-1',index_col=0)
df = df.dropna(subset=['income'])

In [ ]:
df.head()

Linear regression is a methodology that can be used for either prediction or causal analysis depending on how it is designed. For now we will use it for prediction. 

The way it works is that we assume that there is a linear expression of one variable $y$ called the 'dependent' or 'outcome', based on other ones called the 'independent variables' $X$ and some unobservables $\varepsilon$

$$y=\beta_0+\beta_1 X+\varepsilon$$


By finding the values of $\beta_0$ and $\beta_1$ that best solve for the group of equations we can find with data for $y$ and $X$, we can then predict what values of $y$ would be if we had some hypothetical values of $X$ not in the data.

While it seems that this form is very restrictive, $X$ can take any transformation of a variable, including polynomials, sines and others that while being written in linear form, don't necessarily generate straight lines. 


We are going to look at the 2016 election data to see if we can predict the share of voters for Trump in a county using the average income for it.

$$y=\beta_0+\beta_1 X+\varepsilon$$

$$\frac{dy}{dX}=\beta_1$$

LaTeX



In [ ]:
# y is trump_share, X is income

reg1 = smf.ols('trump_share ~ income',df).fit()

In [ ]:
reg1.summary()

In [ ]:
#OK, so how do we know what are the betas?

reg1.params

Intercept is $\hat{\beta}_0$ the estimate for $\beta_0$, and income is $\hat{\beta}_1$ the estimate for $\beta_1$

If we wanted to see all predicted values of $y$ if we fed the same values of $X$ to our equation we could do this

In [ ]:
df['preds_reg1']=reg1.predict()

In [ ]:
df.head()

In [ ]:
# We will sort the dataframe by the values of income so we
# can plot easily


df.sort_values(by='trump_share', inplace=True)

df.head()

In [ ]:
(ggplot(df, aes(x='income', y='trump_share'))+
       geom_point(color='steelblue')+
 geom_line(aes(x='income', y='preds_reg1'), color='darkorange' )+
 theme_bw()
       )

This doesn't look like it did very well... maybe we can use a different independent variable like population

In [ ]:
reg2 = smf.ols('trump_share ~ population',df).fit()
reg2.summary()

In [ ]:
df['preds_reg2']=reg2.predict()



In [ ]:
(ggplot(df, aes(x='population', y='trump_share'))+
       geom_point(color='steelblue')+
 geom_line(aes(x='population', y='preds_reg2'), color='darkorange' )+
 theme_bw()
       )

In [ ]:
# Let's look at the distribution of population


(ggplot(df, aes(x='population'))+
geom_histogram(fill='darkorange', bins=100)+
 theme_bw()
)

In [ ]:
np.log(1000)-np.log(100)

In [ ]:
np.log(10000)-np.log(1000)

In [ ]:
(ggplot(df, aes(x=np.log(df['population'])))+
 geom_histogram(fill='darkorange')+
 theme_bw()
)

In [ ]:
#Let's make a new variable and see if it works better

df['lnpop']=np.log(df['population'])

In [ ]:
reg3 = smf.ols('trump_share ~ lnpop',df).fit()
reg3.summary()

In [ ]:
df['preds_reg3']=reg3.predict()


In [ ]:
(ggplot(df, aes(x='lnpop', y='trump_share'))+
       geom_point(color='steelblue')+
 geom_line(aes(x='lnpop', y='preds_reg3'), color='darkorange' )+
 theme_bw()
       )

In [ ]:
#This looks a lot better! I want to check something though

(ggplot(df, aes(x='population', y='trump_share'))+
       geom_point(color='steelblue')+
 geom_line(aes(x='population', y='preds_reg3'), color='darkorange')+
 theme_bw()
       )

In [ ]:
#we can assess the fit with the R squared, the closest it is to
#to one, the better the fit

print('reg with just income R squared is:', reg1.rsquared)
print('reg with just population R squared is:', reg2.rsquared)
print('reg with just log population R squared is:', reg3.rsquared)



In [ ]:
# so we see we are doing better. Let's do one more

reg4 = smf.ols('trump_share ~ income+lnpop',df).fit()
reg4.summary()

In [ ]:
df['preds_reg4']=reg4.predict()

In [ ]:
(ggplot(df, aes(x='income', y='trump_share'))+
       geom_point(color='steelblue')+
 geom_line(aes(x='income', y='preds_reg4'), color='darkorange' )+
 theme_bw()
       )

In [ ]:
(ggplot(df, aes(x='lnpop', y='trump_share'))+
       geom_point(color='steelblue')+
 geom_line(aes(x='lnpop', y='preds_reg4'), color='darkorange' )+
 theme_bw()
       )

In [ ]:
regdf=reg4.conf_int()
regdf

In [ ]:
regdf['betas']=reg4.params
regdf

In [ ]:
regdf.reset_index(inplace=True)
regdf.columns=['Variable','Lower_Bound', 'Upper_Bound', 'Beta']

In [ ]:
regdf

In [ ]:
#We could do a quick visualization of this for interpretation!
(ggplot(regdf, aes(y='Beta', x='Variable', label='Beta'))+
 
 
 geom_point(size=14, color='steelblue')+
 geom_errorbar(ymin=regdf['Lower_Bound'], ymax=regdf['Upper_Bound'],
              color='steelblue')+
 geom_text(format_string='{:.4f}', color='white', size=5)+
 ylim([-1, 1.4])+
 geom_hline(yintercept=0, linetype='dashed', color='darkorange')+
 coord_flip()+
 theme_bw()

)



The things that are difficult about this:


* Specifying a good relationship is hard!
* We don't know what other variables could be relevant!
* If we add everything but the kitchen sink, the R square goes up...but it doesn't mean that the model is better



The things that are great!

* Interpretation is simple!!! 

For 

$$y=\beta_0+\beta_1 X+\varepsilon$$


$$\beta_1=\frac{dy}{dx}$$

We can have statistical tests around our parameters!

